In [62]:
import pandas as pd

data = [
    ("Vardas1", 0, 165, 55),
    ("Vardas2", 1, 183, 95),
    ("Vardas3", 0, 171, 60),
    ("Vardas4", 1, 194, 102),
    ("Vardas5", 1, "?", 80),
    ("Vardas6", 1, 185, 90),
]

In [63]:
# pirmas
df = pd.DataFrame(data, columns=['vardas', 'lytis', 'ugis', 'svoris'])
filteredTable = df.drop(df[df['ugis'] == "?"].index)
# print(filteredTable)
# print('---- \n')

print(f"all average {filteredTable['ugis'].mean()}")
print(f"all median {filteredTable['ugis'].median()}")
print('\n ---- \n')

women = filteredTable[filteredTable['lytis'] == 0]
print(f"women average {women['ugis'].mean()}")
print(f"women median {women['ugis'].median()}")
print('\n ---- \n')

men = filteredTable[filteredTable['lytis'] == 1]
print(f"men average {men['ugis'].mean()}")
print(f"men median {men['ugis'].median()}")

all average 179.6
all median 183.0

 ---- 

women average 168.0
women median 168.0

 ---- 

men average 187.33333333333334
men median 185.0


In [64]:
#antra
from scipy.spatial import distance
import numpy as np

def norm(column):
    return (column - column.mean())/column.std()

def denorm(norm_var, column):
    return norm_var * column.std() + column.mean()

def metrikos_skaiciavimas(kintamieji, metrikos):
    return (1/np.sum(1/metrikos)*np.sum(kintamieji/metrikos))

norm_lytis = norm(df['lytis'])
# print(norm_lytis)

norm_ugis = norm(filteredTable['ugis'])
# print(norm_ugis)

norm_svoris = norm(df['svoris'])
# print(norm_svoris)



atstumas_e = np.sqrt((norm_lytis[4] - norm_lytis)**2+(norm_svoris[4]-norm_svoris)**2)
del atstumas_e[4]
# print(atstumas_e)

ugis_norm_e = metrikos_skaiciavimas(norm_ugis, atstumas_e)
ugis_e = denorm(ugis_norm_e, filteredTable['ugis'])
print(f"ugis euclidean - {ugis_e}")



atstumas_m = np.abs(norm_lytis[4] - norm_lytis)+np.abs(norm_svoris[4]-norm_svoris)
del atstumas_m[4]
# print(atstumas_m)

ugis_norm_m = metrikos_skaiciavimas(norm_ugis, atstumas_m)
ugis_m = denorm(ugis_norm_m, filteredTable['ugis'])
print(f"ugis manhattan - {ugis_m}")


atstumas_c = np.amax(
                np.stack(
                    (np.abs(norm_lytis[4] - norm_lytis), np.abs(norm_svoris[4]-norm_svoris)
                ), axis=-1), 
            axis=1).tolist()

del atstumas_c[4]
atstumas_c = pd.DataFrame(atstumas_c)[0]
# print(atstumas_c)

ugis_norm_c = metrikos_skaiciavimas(norm_ugis, atstumas_c)
print(ugis_norm_c)
ugis_c = denorm(ugis_norm_c, filteredTable['ugis'])
print(f"ugis Čebyšavo  - {ugis_c}")



ugis euclidean - 183.04910652784145
ugis manhattan - 183.81314372942035
0.08303873239059639
ugis Čebyšavo  - 180.56052520951172


In [65]:
#trecias
filteredTable = filteredTable.astype({"ugis": np.int64})

avg_lyt = filteredTable['lytis'].mean()
avg_ugis = filteredTable['ugis'].mean()
avg_svoris = filteredTable['svoris'].mean()
print(f"average lyt {avg_lyt}")
print(f"average ugis {avg_ugis}")
print(f"average svoris {avg_svoris}")

corr = filteredTable.corr()
corr_lyt = corr['ugis'][0]
corr_svoris = corr['ugis'][2]
# print(corr_lyt)
# print(corr_svoris)
# print(df['svoris'][4])
atkurimas_svoris = corr_svoris * (df['svoris'][4] - avg_svoris)
# print(atkurimas_svoris)
atkurimas_lyt = corr_lyt * (df['lytis'][4] - avg_lyt)
# print(atkurimas_lyt)
atsakymas = avg_ugis + (1/(abs(corr_lyt) + abs(corr_svoris)) * (atkurimas_svoris + atkurimas_lyt)) 
# atsakymas = avg_ugis + (1/ abs(corr_svoris) * atkurimas_svoris )
print(atsakymas)


average lyt 0.6
average ugis 179.6
average svoris 80.4
179.58884652676338


In [77]:
rekomendacijosRaw = [
    ("A. Klasikinė suknelė", 1, 1, 1, 0, 0, 0, 1),
    ("B. Retro suknelė", 0, 1, 0, 0, 1, 0, 0),
    ("C. Klasikinis kostiumas su kelnėmis", 0, 0, 0, 1, 0, 1, 1),
    ("D. Klasikinis kostiumas su sijonu", 0, 0, 1, 0, 0, 0, 0),
    ("E. Kelnės", 0, 1, 0, 1, 0, 0, 0),
    ("F. Sijonas", 1, 0, 0, 0, 1, 1, 0)
]

rekomendacijosFull = pd.DataFrame(rekomendacijosRaw, columns=['prekes', '1', '2', '3', '4', '5', '6', '7'])
rekomendacijos = rekomendacijosFull.drop("prekes", axis=1)
print(rekomendacijos)


   1  2  3  4  5  6  7
0  1  1  1  0  0  0  1
1  0  1  0  0  1  0  0
2  0  0  0  1  0  1  1
3  0  0  1  0  0  0  0
4  0  1  0  1  0  0  0
5  1  0  0  0  1  1  0


In [80]:
#ketvirtas
from scipy.spatial.distance import cdist
dist = cdist(rekomendacijos, rekomendacijos, 'euclid')
# print(dist)

for i in range(len(dist)):
    dist[i][i] = np.inf
# print(dist)
print()

shortest_dist = []
for index, preke in enumerate(rekomendacijos["7"]):
    if preke == 1:
        shortest_dist_in_row_index = dist[index].argmin()
        shortest_dist_in_row = dist[index].min()
        row_name = rekomendacijosFull["prekes"][shortest_dist_in_row_index]

        # print(shortest_dist_in_row)
        shortest_dist.append((row_name, shortest_dist_in_row))
        # print(index)

shortest_dist = pd.DataFrame(shortest_dist, columns=['prekes', 'atstumas'])
shortest_dist = shortest_dist.loc[shortest_dist.groupby('prekes').atstumas.idxmin()]
print(shortest_dist)


                              prekes  atstumas
0  D. Klasikinis kostiumas su sijonu  1.732051
1                          E. Kelnės  1.732051


In [ ]:
# penkta
filmaiRaw = [
    ("A. Klasikinė suknelė", 1, 1, 1, 0, 0, 0, 1),
    ("B. Retro suknelė", 0, 1, 0, 0, 1, 0, 0),
    ("C. Klasikinis kostiumas su kelnėmis", 0, 0, 0, 1, 0, 1, 1),
    ("D. Klasikinis kostiumas su sijonu", 0, 0, 1, 0, 0, 0, 0),
    ("E. Kelnės", 0, 1, 0, 1, 0, 0, 0),
    ("F. Sijonas", 1, 0, 0, 0, 1, 1, 0)
]

rekomendacijosFull = pd.DataFrame(rekomendacijosRaw, columns=['prekes', '1', '2', '3', '4', '5', '6', '7'])
rekomendacijos = rekomendacijosFull.drop("prekes", axis=1)
print(rekomendacijos)


In [145]:
#sesta
from scipy import special

def erfc_value(array, value):
    return abs(value - array.mean())/array.std()

def anomaly_criteria(array):
    return 1/(2 * len(array))

A = np.asarray([15, 17, 16, 16, 18, 17, 15, 30, 18])
B = np.asarray([32.88, 40.81, 85.32, 53.15, 70.91, 30.84, 64.86, 54.31])
erf_array = special.erfc(A)
# erf_array = special.erfc(B)
# print(erf_array)

# print(B.mean())
# print(B.std())
# print(erfc(B, 85.32))
# print(anomaly_criteria(B))
val = special.erfc(erfc(B, 85.32))

anomalies = True
dataset = A
while anomalies:
    anomalies = False
    for index, i in enumerate(dataset):
        if special.erfc(erfc_value(dataset, i)) < anomaly_criteria(dataset):
            dataset = np.delete(dataset, index)
            print(i)
            anomalies = True
            break

print('liko: ')
print(dataset)



30
15
15
16
16
liko: 
[17 18 17 18]


In [148]:
C = np.asarray([-2, -1, 0, 1, 3, 5, 7, 8, 9, 10])
q25 = np.quantile(A, 0.25)
q75 = np.quantile(A, 0.75)
print(q25, q75)
condition1 = q25 - 1.5 * (q75 - q25)
condition2 = q75 + 1.5 * (q75 - q25)
print(condition1, condition2)

print('isimtys: ')
dataset = A
for index, i in enumerate(dataset):
    if i < condition1 or i > condition2:
        dataset = np.delete(dataset, index)
        print(i)

print('liko: ')
print(dataset)


16.0 18.0
13.0 21.0
isimtys: 
30
liko: 
[15 17 16 16 18 17 15 18]
